In [1]:
import re
import json
import ssl
import pandas as pd
import urllib.request

### Trello_Analyzer 使用方法：
    - Step1: 获取个人 Trello 上的 key 和 token 等请求参数，获取地址:https://developers.trello.com/reference/#api-key-tokens
    - Step2: 在下方 getDoneList 函数中填写日期（也就是list 名称），如 "Done[20191209]", 则是抓取12月9号之后的所有 done_list
    - Step3: 把所有 list 都 concat 到一张表上，就可以开始分析数据了

In [ ]:
# 设置参数
g_token = "your_g_token"
g_app_key = "your_g_app_key"
g_user_id = "your_g_user_id"
g_board_id = "your_g_board_id"

workload_pattern = u'[(（]\s*(\d+(?:\.\d+)?)\s*(pt|PT|pT|Pt)\s*[)）]' # 正则匹配 pt 等
requirement_pattern = u'[\[【［]\s*(.*)\s*[】］\]]\s*' 


In [ ]:
# 设置参数
g_token = "c68bb70c102d863b3d96bc5b8901ff7689dfcba68f040b4fb3d56e9b2f8dbab8"
g_app_key = "e70d4c831ca1c79442f7ef7a68aeb9e3"
g_user_id = "5c16ff1d6e249c62d057987f"

# g_board_id = "5ac3159ed146b0793ff30206" # ThunderDB
# g_board_id = "5b640e00c8b518181d80fa46" # Rating system
g_board_id = '5d2b3dbf9c7fb38e43f6b682' # sh-bank
# g_board_id =  '5d5b5a47ce0b537663499e30' # 数据中台

workload_pattern = u'[(（]\s*(\d+(?:\.\d+)?)\s*(pt|PT|pT|Pt)\s*[)）]' # 正则匹配 pt 等
requirement_pattern = u'[\[【［]\s*(.*)\s*[】］\]]\s*' 


In [3]:
def basic_replace(url): # 替换 url 等主要参数
    url = url.replace("_APP_KEY_", g_app_key)
    url = url.replace("_TOKEN_", g_token)
    return url

def do_request(url): 
    context = ssl._create_unverified_context()
    request = urllib.request.Request(url)
    content = None
    flag = False

    for i in range(3):
        try:
            response = urllib.request.urlopen(request, context= context) # 会验证一次 SSL 证书, 当目标使用的是自签名的证书时就会报 SSL 错误
            content = response.read().decode()
            flag = True
            break
        except urllib.request.HTTPError as e:
            print("http error: " + str(e))
        except Exception as e:
            print("error: " + str(e))

    if not flag:
        sys.exit(-1)

    if content is None:
        return []
    else:
        return json.loads(content)
    
def get_card_name_by_pattern(card_name, pattern):
    return re.findall(pattern, card_name, re.S | re.U) # 搜索字符串，以列表类型返回全部能匹配的子串

def fetch_board_by_user():
    url = 'https://api.trello.com/1/members/_USERID_/boards?key=_APP_KEY_&token=_TOKEN_'
    url = basic_replace(url)
    url = url.replace("_USERID_", g_user_id)
    body = do_request(url)
    board_info = {}

    for item in body:
        board_info[item['shortLink']] = item['name']

    return board_info

def insertStr(str_name): # 匹配"[]"里的日期
    
    if "[" in str_name: 
        str_list = list(str_name) 
        str_pos = str_list.index('[') 
        str_list.insert(str_pos, '\\') 
        str_2 = "".join(str_list)
    
    else:
        str_2 = str_name
    
    return str_2



In [4]:
def card_action(card_id):
    url = 'https://api.trello.com/1/cards/jwoPoqiY/actions?&key=_APP_KEY_&token=_TOKEN_'
    url = basic_replace(url)
    url = url.replace('_CARDID_', card_id)
    body = do_request(url)
    return body,url


In [5]:
def fetch_list_id_by_board(list_pattern): # list_pattern 作为参数要输入 list name 
    url = 'https://api.trello.com/1/boards/_BOARDID_/lists?key=_APP_KEY_&token=_TOKEN_'
    url = basic_replace(url)
    url = url.replace("_BOARDID_", g_board_id)
    body = do_request(url) # 返回特定 board 里所有的 list 信息
    list_info = []
    list_name = insertStr(list_pattern)
    
    for item in body:
        
        if re.search(list_name, item["name"], re.I) is not None: # re.search 在一个字符串中搜索匹配正则表达式的第一个位置，返回match对象
            list_info.append({'id': item['id'], 'name': item['name']}) 
            

    return list_info # 返回 list name 和 list_id


In [6]:
def fetch_members_by_board(): # 拿到所有成员的名称和 id
    url = 'https://api.trello.com/1/boards/_BOARDID_/members?key=_APP_KEY_&token=_TOKEN_'
    url = basic_replace(url)
    url = url.replace("_BOARDID_", g_board_id)
    body = do_request(url)
    all_members_info = {}

    for item in body:
        all_members_info[item['id']] = item['fullName']

    return all_members_info # 返回 id 和 member name

In [7]:
def fetch_cards_by_list_id(list_id): #  可以通过上面的 fetch_list_id_by_board 可以获取 list_id
    url = 'https://api.trello.com/1/lists/_LISTID_/cards?key=_APP_KEY_&token=_TOKEN_'
    url = basic_replace(url)
    url = url.replace("_LISTID_", list_id) 
    body = do_request(url)

    return body # 用 list_id 获取 cards 的所有信息

In [8]:
def fetch_name_by_checklist_id(checklist): 
    url = 'https://api.trello.com/1/checklists/_ID_/checkItems?key=_APP_KEY_&token=_TOKEN_'
    url = basic_replace(url)
    url = url.replace("_ID_", checklist) 
    body = do_request(url)
    checklist_name = []
    for item in body:
        checklist_name.append(item['name'])
        
    return checklist_name 

In [9]:
def get_cards_info(list_id, board_members):

    available_cards_info = []
    all_cards_info = fetch_cards_by_list_id(list_id)

    for item in all_cards_info:
        card_info = { 
            'card_name': item['name'],
            'card_id': item['id'],
            'member_name': None,
            'member_id': None,
            'label_name': 'None',
            'label_id': None,
            'plan_hours': 0,
            'actual_hours': 0, 
            'description' : None,
            'checklist' : None,
            'subproject':None
        }
        
        subproject = item['name'].split('--')

        if len(subproject) > 0 and subproject is not None:
            card_info['subproject'] = subproject[0]
        
        if len(item['idChecklists']) > 0 and item['idChecklists'][0] is not None:
            card_info['checklist'] = fetch_name_by_checklist_id(item['idChecklists'][0])
        
        idMembers = item['idMembers'] # 让 cards_info get 到 member 的信息
        if len(idMembers) > 0 and idMembers[0] is not None:
            member_id = idMembers[0]
            card_info['member_id'] = member_id
            card_info['member_name'] = board_members[member_id]

        labels = item['labels']

        if len(labels) > 0 and labels[0] is not None: # 处理卡片的 labels
            label = labels[0]
            card_info['label_id'] = label['id']
            card_info['label_name'] = list(value['name'] for value in item['labels'])

        card_hours = get_card_name_by_pattern(item['name'], workload_pattern)
        
        if len(card_hours) == 2:
            card_info['actual_hours'] = float(card_hours[1][0]) * 2
            card_info['plan_hours'] = float(card_hours[0][0]) * 2
        else:
            try:
                card_info['plan_hours'] = float(card_hours[0][0]) * 2
                card_info['actual_hours'] = float(card_hours[0][0]) * 2
            except IndexError as e:
                print("IndexError: " + item['name'])

        
        card_info['description'] = item['desc']
        available_cards_info.append(card_info)
                       
    return available_cards_info

In [10]:
def produce_dataframe(list_name):
    board_members = fetch_members_by_board()
    one_week_list = get_cards_info(fetch_list_id_by_board(list_name)[0]['id'], board_members)
    df = pd.DataFrame(one_week_list, columns=[
                                              'label_name', 
                                              'card_name', 
                                              'subproject',
                                              'member_name',
                                              'plan_hours',
                                              'actual_hours',
                                              'checklist',
                                              'description',
                                              'card_id',
                                              'label_id',
                                              'member_id',
                                             ])
    return df


In [11]:
def getDoneList(until_date): # 选择抓取范围
    
    done_list = []
    for i in fetch_list_id_by_board('Done'):
        done_list.append(i['name'])
        
    return done_list[:done_list.index(until_date)+1]

In [25]:
done_list = getDoneList("Done[20191209]")
done_list

## 可以在这里先run一下 done_list 看是不是需要抓取的列表

['Done[20191216]',
 'Done[20191213]',
 'Done[20191212]',
 'Done[20191211]',
 'Done[20191210]',
 'Done[20191209]']

In [ ]:
dfs = []
for done_date in done_list:
    result = produce_dataframe('{}'.format(done_date))
    dfs.append(result)
    
result = pd.concat(dfs,join='outer') ## 整合到一张表里

In [19]:
result.to_csv("sh_bank_20191209.csv",encoding='utf_8_sig') ## 存储到本地

In [23]:
## 去掉某位 member 的数据
result_remove1 = total_statistic_result[total_statistic_result['member_name']!='liwan'] # remove liwan 
result_remove2 = result_remove1[result_remove1['member_name']!='严龙祥']# remove 严龙祥  
sum(result_remove2['actual_hours'])

In [ ]:

# 本地的多表整合 

# files = os.listdir('/Users/lichenxi/venv/CovenantSQL/Trello Analyzer')
# fn = files[0]
# df = pd.read_csv(os.path.join('/Users/lichenxi/venv/CovenantSQL/Trello Analyzer', fn))

# dfs = []
# for fn in files:
#     try:
#         df = pd.read_csv(os.path.join('/Users/lichenxi/venv/CovenantSQL/Trello Analyzer/', fn))
#         df.set_index("Unnamed: 0", inplace=True)
#         dfs.append(df)
#     except Exception as e:
#         print('Error loading: ', fn)
#         print(e)
